In [19]:
import pandas as pd
from gensim.corpora.dictionary import Dictionary
import re
import numpy as np
from nltk.corpus import stopwords
import pymorphy2
from gensim.models import LdaModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from razdel import tokenize
from gensim.test.utils import datapath
import itertools

%matplotlib inline

In [20]:
from sklearn.pipeline import Pipeline

In [21]:
news = pd.read_csv("articles.csv")
users = pd.read_csv("users_articles.csv")



In [26]:
class TextCleanerTokenizer:
    
    def __init__(self, column: str):
        self.column = column
        
    def fit(self, text, y=None):
        return self
    
    def cleaner(self, text):
        if not isinstance(text, str):
            text = str(text)
    
        text = text.lower()
        text = text.strip('\n').strip('\r').strip('\t')
        text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

        text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
        text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n|n", ' ', text)
        text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

        return text
    
    def lemmatization(self, text):
        cache = {}
        stopword_ru = stopwords.words('russian')
        morph = pymorphy2.MorphAnalyzer()
        if not isinstance(text, str):
            text = str(text)
        tokens = list(tokenize(text))
        words = [_.text for _ in tokens]

        words_lem = []
        for w in words:
            if w[0] == '-':
                w = w[1:]
            if len(w)>1: 
                if w in cache:
                    words_lem.append(cache[w])
                else: 
                    temp_cach = cache[w] = morph.parse(w)[0].normal_form
                    words_lem.append(temp_cach)

        words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru]

        return words_lem_without_stopwords
    
    def transform(self, X):
        X[self.column] = X[self.column].apply(lambda x: self.cleaner(x), 1)
        X[self.column] = X[self.column].apply(lambda x: self.lemmatization(x), 1)
        return X
    
#     def fit_transform(self, X, y=None):
#         data = self.fit(X)
#         return self.transform(X)



In [27]:
a = Pipeline([
    ('cleaner', TextCleanerTokenizer(column='title'))
])

In [ ]:
q = a.fit_transform(news)

In [ ]:
q